# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [32]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [33]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [34]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.10/warnings.py", line 538, in <module>
  File "/usr/lib/python3.10/warnings.py", line 208, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.10/warnings.py", line 224, in _setoption
    import re
  File "/usr/lib/python3.10/re.py", line 145, in <module>
    class RegexFlag(enum.IntFlag):
  File "/usr/lib/python3.10/enum.py", line 200, in __new__
    del classdict[name]
KeyboardInterrupt


In [35]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [36]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# Step 2: Load the Traffic Violations Dataset

In [37]:
df.shape

(104, 2)

In [38]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [39]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [40]:
for i in range(0, len(df)):

    violation = df['المخالفة'].iloc[i]
    fine = df['الغرامة'].iloc[i]

    markdown_content = f"# {violation}\n\n"
    markdown_content += f"{fine}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [41]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [43]:
!pip install transformers

In [14]:
!pip install transformers


In [52]:
embedding_function = SentenceTransformerEmbeddings(model_name="aubmindlab/bert-base-arabertv02")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [53]:
PROMPT_TEMPLATE = """
استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: {context}
السؤال: {question}
إجابتك:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [54]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [55]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [56]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=5)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [61]:
query = "ماهي الغرامة على القيادة بدون رخصة؟"
similarity_search_results = db.similarity_search_with_score(query, k=4)
print(similarity_search_results[0][1])
print(similarity_search_results[1][1])
print(similarity_search_results[2][1])
print(similarity_search_results[3][1])

182.7650909423828
182.7650909423828
182.7650909423828
182.7650909423828


In [38]:
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


In [48]:
df.tail()

,المخالفة,الغرامة
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال
103,المجازفة بعبور الأودية والشعاب أثناء جريانها.,الغرامة المالية 5000 - 10000 ريال


In [50]:
df.sample(10)

,المخالفة,الغرامة
13,عدم تقديم المركبة للفحص الفني الدوري.,الغرامة المالية 150 - 300 ريال
19,التجمهر في موقع الحادث.,الغرامة المالية 150 - 300 ريال
65,سير المركبة بلا لوحة أمامية.,الغرامة المالية 1000 - 2000 ريال
73,النزول أو الركوب أثناء سير المركبات.,الغرامة المالية 1000 - 2000 ريال
23,رمي أجسام خارج المركبة.,الغرامة المالية 300 - 500 ريال
17,عدم ترك السائق مسافة كافية بين مركبته والمركبة...,الغرامة المالية 150 - 300 ريال
27,عدم التقيد بتنظيمات السير عند تقاطعات الطرق.,الغرامة المالية 300 - 500 ريال
85,تجاوز حافلات النقل المدرسي عند توقفها للتحميل ...,الغرامة المالية 3000 - 6000 ريال
31,عدم استخدام مقاعد الأمان المخصصة للأطفال.,الغرامة المالية 300 - 500 ريال
81,إجراء أي تعديل أو إضافة على جسم المركبة أو هيك...,الغرامة المالية 1000 - 2000 ريال


In [56]:
pd.set_option('display.max_rows', None)
display(df)

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
5,وقوف المركبة في أماكن غير مخصصة للوقوف.,الغرامة المالية 100 - 150 ريال
6,عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في...,الغرامة المالية 100 - 150 ريال
7,عدم استخدام إشارة الالتفاف عند التحول لليمين أ...,الغرامة المالية 150 - 300 ريال
8,الرجوع إلى الخلف في الطريق العام لمسافة تزيد ع...,الغرامة المالية 150 - 300 ريال
9,قيام سائق الدراجة الآلية أو العادية - أو ما في...,الغرامة المالية 150 - 300 ريال


In [46]:
rows = df[df['المخالفة'] == 'استخدام لوحات غير عائدة للمركبة.']
print(rows)


                             المخالفة                            الغرامة
101  استخدام لوحات غير عائدة للمركبة.  الغرامة المالية 5000 - 10000 ريال


In [58]:
response = query_rag("ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟")
response

Prompt after formatting:

استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: <h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>
السؤال: ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟
إجابتك:


> Finished chain.


{'context': '<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 'question': 'ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟',
 'text': 'الغرامة المالية 1000 - 2000 ريال.'}

In [60]:
def inference_pipeline():
    user_query = input("سؤالك؟")
    response = query_rag(user_query)
    print("الجواب:", response)


inference_pipeline()

سؤالك؟ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟"


Prompt after formatting:

استنادًا إلى المعلومات التالية فقط، قم بالإجابة على السؤال:
المعلومات: <h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

<h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>
السؤال: ماهي الغرامة على قيادة المركبة قبل الحصول على رخصة؟"
إجابتك:


> Finished chain.
الجواب: {'context': '<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها.</h1>\n<p>الغرامة المالي